# Day14_1: 차원 축소 (Dimensionality Reduction) - 정답 노트북

---

In [ ]:
# 필요한 라이브러리
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

## Q1. 데이터 표준화하기 (난이도: 1/5)

**문제**: Iris 데이터를 StandardScaler로 표준화하고, 표준화 후 평균과 표준편차를 출력하세요.

In [ ]:
# 정답 코드
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 결과 출력
print("표준화 전:")
print(f"  평균: {X.mean(axis=0).round(2)}")
print(f"  표준편차: {X.std(axis=0).round(2)}")

print("\n표준화 후:")
print(f"  평균: {X_scaled.mean(axis=0).round(6)}")
print(f"  표준편차: {X_scaled.std(axis=0).round(6)}")

In [ ]:
# 테스트
assert X_scaled.shape == X.shape, "형태가 동일해야 함"
assert np.allclose(X_scaled.mean(axis=0), 0, atol=1e-10), "평균은 0에 가까워야 함"
assert np.allclose(X_scaled.std(axis=0), 1, atol=1e-10), "표준편차는 1에 가까워야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- StandardScaler는 각 특성의 평균을 0, 표준편차를 1로 변환
- `fit_transform()`은 학습과 변환을 동시에 수행

**핵심 개념**:
- 표준화 공식: z = (x - mean) / std
- PCA 전 표준화는 필수! (스케일이 다르면 분산 계산이 왜곡됨)

**대안**:
- `MinMaxScaler`: 0~1 범위로 정규화
- `RobustScaler`: 이상치에 강건한 스케일링

**흔한 실수**:
- fit()만 하고 transform() 안 함
- 테스트 데이터에 fit_transform() 사용 (정보 누출)

**실무 팁**:
- 학습 데이터로 fit(), 테스트 데이터는 transform()만 사용

---

## Q2. PCA 2차원 변환 (난이도: 2/5)

**문제**: 표준화된 Iris 데이터를 PCA로 2차원으로 축소하고, 설명된 분산 비율을 출력하세요.

In [ ]:
# 정답 코드
from sklearn.decomposition import PCA

# PCA 적용 (2차원)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 설명된 분산 출력
print(f"PCA 결과 형태: {X_pca.shape}")
print("\n설명된 분산 비율:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%)")

cumulative = sum(pca.explained_variance_ratio_)
print(f"\n누적 설명 분산: {cumulative:.4f} ({cumulative*100:.2f}%)")

In [ ]:
# 테스트
assert X_pca.shape == (150, 2), "2차원으로 축소되어야 함"
assert len(pca.explained_variance_ratio_) == 2, "2개의 주성분"
assert sum(pca.explained_variance_ratio_) > 0.9, "90% 이상 설명해야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- PCA(n_components=2)로 2개의 주성분만 추출
- explained_variance_ratio_로 각 PC의 설명 분산 확인

**핵심 개념**:
- PC1은 데이터 분산의 약 73%를 설명
- PC1 + PC2 = 약 96%, 4차원 데이터를 2차원으로 압축해도 정보 손실 4%

**대안**:
- `n_components=0.95`: 95% 분산을 설명하는 최소 성분 수 자동 선택
- `svd_solver='randomized'`: 대용량 데이터에서 빠른 근사 계산

**흔한 실수**:
- 표준화 없이 PCA 적용 (스케일 영향)
- n_components를 데이터 차원보다 크게 설정

**실무 팁**:
- 시각화 목적이면 2~3개, 전처리 목적이면 누적 90~95% 기준으로 선택

---

## Q3. PCA 시각화 (난이도: 2/5)

**문제**: PCA 변환된 Iris 데이터를 px.scatter()로 시각화하세요.

In [ ]:
# 정답 코드
import plotly.express as px
import pandas as pd

target_names = iris.target_names
y = iris.target

# DataFrame 생성
df_pca = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Species': [target_names[i] for i in y]
})

# 시각화
fig = px.scatter(
    df_pca, x='PC1', y='PC2', color='Species',
    title=f'Iris PCA (설명 분산: {sum(pca.explained_variance_ratio_)*100:.1f}%)',
    labels={
        'PC1': f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
        'PC2': f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)'
    },
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(marker=dict(size=10, opacity=0.7))
fig.show()

### 풀이 설명

**접근 방법**:
- PCA 결과를 DataFrame으로 변환
- Species 컬럼 추가로 색상 구분
- px.scatter()로 2D 산점도 생성

**핵심 개념**:
- 축 라벨에 설명 분산 표시 -> 정보량 시각화
- Setosa는 명확히 분리, Versicolor/Virginica는 일부 중첩

**대안**:
- `px.scatter_3d()`: 3차원 시각화
- `hover_data`: 추가 정보 표시

**흔한 실수**:
- target을 그대로 사용 (0, 1, 2 대신 이름 사용 권장)
- 축 라벨에 설명 분산 미표시

**실무 팁**:
- 시각화 시 항상 설명 분산 비율을 함께 표시하여 정보 손실량 명시

---

## Q4. Scree Plot 그리기 (난이도: 3/5)

**문제**: Wine 데이터에 대해 모든 주성분의 설명 분산을 막대그래프로, 누적 분산을 선그래프로 그리세요.

In [ ]:
# 정답 코드
from sklearn.datasets import load_wine
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

wine = load_wine()
X_wine = wine.data

# 표준화
scaler_wine = StandardScaler()
X_wine_scaled = scaler_wine.fit_transform(X_wine)

# 전체 PCA
pca_wine = PCA()
pca_wine.fit(X_wine_scaled)

# 데이터 준비
explained_var = pca_wine.explained_variance_ratio_ * 100
cumulative_var = np.cumsum(explained_var)
components = [f'PC{i+1}' for i in range(len(explained_var))]

# Scree Plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 개별 분산 (막대)
fig.add_trace(
    go.Bar(
        x=components,
        y=explained_var,
        name='개별 설명 분산',
        marker_color='steelblue'
    ),
    secondary_y=False
)

# 누적 분산 (선)
fig.add_trace(
    go.Scatter(
        x=components,
        y=cumulative_var,
        name='누적 설명 분산',
        mode='lines+markers',
        marker_color='coral',
        line=dict(width=3)
    ),
    secondary_y=True
)

# 90% 기준선
fig.add_hline(y=90, line_dash="dash", line_color="green",
              annotation_text="90%", secondary_y=True)

fig.update_layout(
    title='Wine Scree Plot',
    xaxis_title='주성분',
    yaxis_title='개별 분산 (%)',
    yaxis2_title='누적 분산 (%)'
)
fig.show()

# 90% 달성 지점
n_90 = np.argmax(cumulative_var >= 90) + 1
print(f"90% 분산 설명: {n_90}개 주성분 필요")

### 풀이 설명

**접근 방법**:
- `PCA()` (n_components 미지정)으로 모든 주성분 계산
- `np.cumsum()`으로 누적 분산 계산
- `make_subplots(secondary_y=True)`로 이중 축 생성

**핵심 개념**:
- Scree Plot: 엘보우 지점에서 주성분 수 결정
- 누적 90% 기준: 정보 손실 10% 이하

**대안**:
- Kaiser 규칙: 고유값 > 1인 성분만 (표준화 데이터)
- 교차 검증으로 다운스트림 태스크 성능 기준

**흔한 실수**:
- 백분율 변환 안 함 (0~1 vs 0~100)
- 누적 분산 계산 실수

**실무 팁**:
- 시각화와 수치 기준을 함께 사용하여 최적 성분 수 결정

---

## Q5. t-SNE 시각화 (난이도: 3/5)

**문제**: Iris 데이터를 t-SNE로 2차원 변환 후 시각화하세요.

In [ ]:
# 정답 코드
from sklearn.manifold import TSNE

# t-SNE 적용
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

# DataFrame 생성
df_tsne = pd.DataFrame({
    'TSNE1': X_tsne[:, 0],
    'TSNE2': X_tsne[:, 1],
    'Species': [target_names[i] for i in y]
})

# 시각화
fig = px.scatter(
    df_tsne, x='TSNE1', y='TSNE2', color='Species',
    title='Iris t-SNE (perplexity=30)',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(marker=dict(size=10, opacity=0.7))
fig.show()

### 풀이 설명

**접근 방법**:
- `TSNE(n_components=2, perplexity=30, random_state=42)` 설정
- `fit_transform()`으로 변환 (t-SNE는 transform() 단독 불가)

**핵심 개념**:
- t-SNE는 지역 구조를 잘 보존 -> 군집 분리가 더 뚜렷
- perplexity: 각 점이 고려하는 이웃 수 (보통 5~50)

**대안**:
- UMAP: t-SNE보다 빠르고 전역 구조도 보존
- 3D t-SNE: `n_components=3`

**흔한 실수**:
- random_state 미설정 -> 매번 다른 결과
- transform() 단독 호출 시도

**실무 팁**:
- 대용량 데이터는 샘플링 후 t-SNE 적용 (계산 비용 높음)

---

## Q6. perplexity 비교 (난이도: 3/5)

**문제**: perplexity를 [5, 15, 30, 50]으로 변경하며 t-SNE 결과를 2x2 서브플롯으로 비교하세요.

In [ ]:
# 정답 코드
perplexities = [5, 15, 30, 50]

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'perplexity={p}' for p in perplexities]
)

colors = {'setosa': 'blue', 'versicolor': 'green', 'virginica': 'red'}

for idx, perp in enumerate(perplexities):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # t-SNE 변환
    tsne_temp = TSNE(n_components=2, perplexity=perp, random_state=42, n_iter=500)
    X_temp = tsne_temp.fit_transform(X_scaled)
    
    # 각 종별 플롯
    for species_name in target_names:
        mask = [target_names[i] == species_name for i in y]
        fig.add_trace(
            go.Scatter(
                x=X_temp[mask, 0],
                y=X_temp[mask, 1],
                mode='markers',
                name=species_name,
                marker=dict(color=colors[species_name], size=6, opacity=0.7),
                showlegend=(idx == 0)
            ),
            row=row, col=col
        )

fig.update_layout(
    title='t-SNE perplexity 비교',
    height=600,
    legend=dict(x=1.02, y=0.5)
)
fig.show()

### 풀이 설명

**접근 방법**:
- 4개의 perplexity 값에 대해 반복
- `make_subplots(rows=2, cols=2)`로 2x2 그리드
- `row = idx // 2 + 1, col = idx % 2 + 1`로 위치 계산

**핵심 개념**:
- 낮은 perplexity: 지역 구조 강조, 작은 군집
- 높은 perplexity: 전역 구조 강조, 큰 군집

**대안**:
- n_iter 조정: 반복 횟수 (기본 1000)
- learning_rate 조정: 학습률

**흔한 실수**:
- 서브플롯 위치 계산 오류
- showlegend 중복 (첫 번째만 True)

**실무 팁**:
- 데이터 크기에 따라 perplexity 조정: 작은 데이터(5~15), 큰 데이터(30~50)

---

## Q7. SelectKBest 적용 (난이도: 4/5)

**문제**: Wine 데이터에서 SelectKBest로 상위 5개 특성을 선택하고, 각 특성의 F-score를 막대그래프로 시각화하세요.

In [ ]:
# 정답 코드
from sklearn.feature_selection import SelectKBest, f_classif

wine = load_wine()
X_wine = wine.data
y_wine = wine.target
wine_features = wine.feature_names

# SelectKBest 적용
selector = SelectKBest(score_func=f_classif, k=5)
X_kbest = selector.fit_transform(X_wine, y_wine)

# 선택된 특성
selected = np.array(wine_features)[selector.get_support()]
print(f"선택된 특성: {list(selected)}")

# F-score 시각화
df_scores = pd.DataFrame({
    'Feature': wine_features,
    'F-Score': selector.scores_
}).sort_values('F-Score', ascending=True)

fig = px.bar(
    df_scores, x='F-Score', y='Feature', orientation='h',
    title='SelectKBest: F-Score (ANOVA)',
    color='F-Score',
    color_continuous_scale='Blues'
)
fig.update_layout(yaxis=dict(categoryorder='total ascending'))
fig.show()

### 풀이 설명

**접근 방법**:
- `SelectKBest(score_func=f_classif, k=5)`로 ANOVA F-검정 기반 선택
- `selector.get_support()`로 선택된 특성 마스크 획득
- `selector.scores_`로 각 특성의 점수 획득

**핵심 개념**:
- f_classif: 분류 문제용 ANOVA F-검정
- F-score가 높을수록 그룹 간 분산이 큼 (분류에 유용)

**대안**:
- `mutual_info_classif`: 상호 정보량 기반 (비선형 관계 포착)
- `chi2`: 카이제곱 검정 (비음수 데이터)

**흔한 실수**:
- 회귀 문제에 f_classif 사용 (f_regression 사용해야 함)
- score_func 미지정

**실무 팁**:
- 빠른 초기 탐색에 적합, 최종 선택은 모델 기반 방법으로

---

## Q8. RFE 적용 (난이도: 4/5)

**문제**: Logistic Regression을 기반으로 RFE를 적용하여 상위 5개 특성을 선택하고, 특성별 순위를 막대그래프로 시각화하세요.

In [ ]:
# 정답 코드
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# RFE 적용
model = LogisticRegression(max_iter=1000, random_state=42)
rfe = RFE(estimator=model, n_features_to_select=5, step=1)
X_rfe = rfe.fit_transform(X_wine, y_wine)

# 선택된 특성
rfe_features = np.array(wine_features)[rfe.support_]
print(f"선택된 특성: {list(rfe_features)}")

# 순위 시각화
df_rfe = pd.DataFrame({
    'Feature': wine_features,
    'Ranking': rfe.ranking_
}).sort_values('Ranking')

fig = px.bar(
    df_rfe, x='Ranking', y='Feature', orientation='h',
    title='RFE: 특성 순위 (1=선택됨)',
    color='Ranking',
    color_continuous_scale='Reds_r'
)
fig.update_layout(yaxis=dict(categoryorder='total descending'))
fig.show()

### 풀이 설명

**접근 방법**:
- `RFE(estimator=model, n_features_to_select=5)`로 설정
- step=1: 매 반복마다 1개씩 제거 (정밀하지만 느림)
- `rfe.ranking_`: 1=선택됨, 숫자 클수록 먼저 제거됨

**핵심 개념**:
- RFE: 모델을 반복 학습하며 가장 덜 중요한 특성을 제거
- 모델의 coef_ 또는 feature_importances_를 기준으로 중요도 평가

**대안**:
- `RFECV`: 교차 검증으로 최적 특성 수 자동 결정
- step > 1: 빠른 실행 (정밀도 감소)

**흔한 실수**:
- coef_ 없는 모델 사용 (KNN 등)
- n_features_to_select > 전체 특성 수

**실무 팁**:
- 대용량 데이터는 step 값을 높여 속도 향상

---

## Q9. Feature Importance 비교 (난이도: 5/5)

**문제**: Random Forest의 feature_importances_를 추출하고, SelectKBest, RFE와 함께 세 가지 방법의 상위 5개 특성을 비교하세요.

In [ ]:
# 정답 코드
from sklearn.ensemble import RandomForestClassifier

# Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_wine, y_wine)

df_importance = pd.DataFrame({
    'Feature': wine_features,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# 상위 5개
rf_top5 = df_importance.head(5)['Feature'].tolist()

# SelectKBest 상위 5개
kbest_top5 = df_scores.nlargest(5, 'F-Score')['Feature'].tolist()

# RFE 상위 5개
rfe_top5 = list(rfe_features)

# 비교 출력
print("특성 선택 방법 비교 (상위 5개):")
print(f"  SelectKBest: {kbest_top5}")
print(f"  RFE:         {rfe_top5}")
print(f"  RF Importance: {rf_top5}")

# 공통 특성
common = set(kbest_top5) & set(rfe_top5) & set(rf_top5)
print(f"\n모든 방법에서 공통: {list(common)}")

# RF Importance 시각화
fig = px.bar(
    df_importance.sort_values('Importance', ascending=True),
    x='Importance', y='Feature', orientation='h',
    title='Random Forest: Feature Importance',
    color='Importance',
    color_continuous_scale='Greens'
)
fig.show()

### 풀이 설명

**접근 방법**:
- Random Forest 학습 후 `feature_importances_` 추출
- 각 방법의 상위 5개를 리스트로 저장
- 집합(set)의 교집합(&)으로 공통 특성 찾기

**핵심 개념**:
- Feature Importance: 각 특성이 분할에 기여한 정도 (불순도 감소량)
- 여러 방법의 공통 특성 = 가장 신뢰도 높은 특성

**대안**:
- Permutation Importance: 더 신뢰할 수 있는 중요도
- SHAP: 개별 예측에 대한 기여도

**흔한 실수**:
- 고카디널리티 특성에 높은 중요도 (편향)
- 상관 특성 간 중요도 분산

**실무 팁**:
- 여러 방법을 앙상블하여 특성 선택의 신뢰도 향상

---

## Q10. 종합 문제: 차원 축소 파이프라인 (난이도: 5/5)

**문제**: MNIST 데이터에 대해 PCA로 90% 분산 설명 주성분 수를 찾고, 원본 vs 축소 데이터의 분류 정확도를 비교하세요.

In [ ]:
# 정답 코드
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"MNIST 데이터: {X_digits.shape[0]}개 샘플, {X_digits.shape[1]}개 특성")

# Step 1: 표준화
scaler_digits = StandardScaler()
X_digits_scaled = scaler_digits.fit_transform(X_digits)

# Step 2: 전체 PCA로 설명 분산 분석
pca_full = PCA()
pca_full.fit(X_digits_scaled)
cumvar = np.cumsum(pca_full.explained_variance_ratio_)

# 90% 분산 설명 주성분 수
n_90 = np.argmax(cumvar >= 0.90) + 1
print(f"\n90% 분산 설명: {n_90}개 주성분 (64 -> {n_90}, {(1-n_90/64)*100:.1f}% 압축)")

# Step 3: 해당 수로 차원 축소
pca_90 = PCA(n_components=n_90)
X_digits_pca = pca_90.fit_transform(X_digits_scaled)

# Step 4: 분류 정확도 비교
model = LogisticRegression(max_iter=1000, random_state=42)

# 원본 (64차원)
score_original = cross_val_score(model, X_digits_scaled, y_digits, cv=5).mean()

# PCA 축소 (n_90차원)
score_pca = cross_val_score(model, X_digits_pca, y_digits, cv=5).mean()

print(f"\n로지스틱 회귀 교차검증 정확도:")
print(f"  원본 (64차원): {score_original:.4f}")
print(f"  PCA ({n_90}차원): {score_pca:.4f}")
print(f"  차이: {(score_pca - score_original)*100:.2f}%p")

In [ ]:
# 테스트
assert n_90 < 64, "차원이 축소되어야 함"
assert X_digits_pca.shape[1] == n_90, "PCA 결과 차원 확인"
assert score_pca > 0.9, "정확도 90% 이상이어야 함"
print("테스트 통과!")

In [ ]:
# 추가: 차원별 정확도 변화 시각화
n_components_list = [5, 10, 15, 20, 25, 30, 40, 50, 64]
scores = []

for n in n_components_list:
    if n < 64:
        pca_temp = PCA(n_components=n)
        X_temp = pca_temp.fit_transform(X_digits_scaled)
    else:
        X_temp = X_digits_scaled
    score = cross_val_score(model, X_temp, y_digits, cv=5).mean()
    scores.append(score)

fig = px.line(
    x=n_components_list, y=scores,
    markers=True,
    title='PCA 차원 수에 따른 분류 정확도',
    labels={'x': '주성분 수', 'y': '정확도'}
)
fig.add_vline(x=n_90, line_dash="dash", line_color="red",
              annotation_text=f"90% 분산 ({n_90}개)")
fig.show()

### 풀이 설명

**접근 방법**:
1. 데이터 표준화
2. 전체 PCA로 누적 분산 계산
3. `np.argmax(cumvar >= 0.9) + 1`로 90% 달성 지점 찾기
4. 해당 차원으로 PCA 변환
5. cross_val_score()로 원본/축소 비교

**핵심 개념**:
- 64차원 -> 21차원 (약 67% 압축)으로도 정확도 유지/향상 가능
- 차원 축소로 노이즈 제거, 과적합 방지 효과

**대안**:
- `PCA(n_components=0.9)`: 90% 분산 설명 자동 선택
- GridSearchCV로 최적 차원 수 탐색

**흔한 실수**:
- argmax 후 +1 누락 (0-indexed)
- 테스트 데이터에 fit_transform() 사용

**실무 팁**:
- PCA 전처리는 학습 속도 향상 + 과적합 방지에 효과적
- 차원-성능 그래프로 최적점 시각화

---

## 학습 정리

### Part 1: PCA 핵심 요약

| 개념 | 핵심 내용 | 코드 |
|------|----------|------|
| 차원의 저주 | 고차원에서 거리 의미 상실, 과적합 위험 | - |
| PCA 원리 | 분산 최대화 축 찾기, 직교 변환 | `PCA(n_components=k)` |
| 설명 분산 | 각 PC가 원본 정보를 얼마나 보존 | `pca.explained_variance_ratio_` |
| 주성분 선택 | 누적 90%+ 또는 엘보우 규칙 | `np.cumsum()` |
| PCA 로딩 | 원본 특성과 PC의 관계 | `pca.components_` |

### Part 2: t-SNE와 특성 선택 핵심 요약

| 방법 | 특징 | 코드 |
|------|------|------|
| t-SNE | 비선형, 시각화 전용, 지역 구조 보존 | `TSNE(perplexity=30)` |
| SelectKBest | 통계적 필터링 (F-test) | `SelectKBest(f_classif, k=5)` |
| RFE | 모델 기반 재귀적 제거 | `RFE(estimator, n_features_to_select)` |
| Feature Importance | 트리 기반 중요도 | `rf.feature_importances_` |

### 실무 팁

1. **PCA 전 반드시 표준화**: 스케일 차이가 크면 결과 왜곡
2. **t-SNE는 시각화 전용**: 새 데이터 변환 불가, 재현성 위해 random_state 설정
3. **특성 선택 앙상블**: 여러 방법의 공통 특성이 가장 신뢰도 높음
4. **perplexity 튜닝**: 데이터 크기에 따라 조정 (작은 데이터는 작은 값)
5. **교차 검증 필수**: 특성 수에 따른 성능 변화 확인